In [27]:
import requests
import psycopg2
import random
import folium
from folium.plugins import HeatMap
import openrouteservice
import time

# Create synthetic Data

In [20]:
# Bereich für zufällige Breiten- und Längengrade in Berlin Kreuzberg/Friedrichshain
latitude_range = (52.49, 52.52)
longitude_range = (13.40, 13.45)

In [21]:
# Rating-Optionen
ratings = [1, 2, 3, 4, 5]

# Funktion zur Generierung zufälliger Koordinaten innerhalb des angegebenen Bereichs
def generate_random_coordinates():
    latitude = random.uniform(*latitude_range)
    longitude = random.uniform(*longitude_range)
    return latitude, longitude

# Funktion zur Generierung eines zufälligen Ratings
def generate_random_rating():
    return random.choice(ratings)

# OpenRouteService Client initialisieren
client = openrouteservice.Client(key='5b3ce3597851110001cf624842c9deb531d34e0795400d234fd30e06')  # Ersetzen Sie 'YOUR_API_KEY' durch Ihren API-Schlüssel


In [22]:
# Funktion zur Interpolation von Punkten zwischen zwei Koordinaten
def interpolate_points(coord1, coord2, num_points):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    interpolated_points = []
    for i in range(1, num_points + 1):
        lat = lat1 + (lat2 - lat1) * i / (num_points + 1)
        lon = lon1 + (lon2 - lon1) * i / (num_points + 1)
        interpolated_points.append((lat, lon))
    return interpolated_points

# Insert Routes

In [23]:
def insert_route(data, start_lat, start_lon, end_lat, end_lon, rating):
    # Connect to your PostgreSQL database
    conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
    cur = conn.cursor()

    # Rating to weight adjustment mapping
    rating_weight_adjustment = {
        1: -2,
        2: -1,
        3: 0,
        4: 1,
        5: 2
    }

    # Insert route into the routes table with rating
    cur.execute("""
        INSERT INTO routes (start_latitude, start_longitude, end_latitude, end_longitude, rating) 
        VALUES (%s, %s, %s, %s, %s) RETURNING id
        """, (start_lat, start_lon, end_lat, end_lon, rating))
    route_id = cur.fetchone()[0]

    # Function to check if a node exists
    def get_node_id(latitude, longitude):
        cur.execute("SELECT id FROM nodes WHERE latitude = %s AND longitude = %s", (latitude, longitude))
        result = cur.fetchone()
        return result[0] if result else None

    # Insert nodes and route_nodes into the database
    sequence = 0
    coordinates = data

    # List to store node IDs in the order of the route
    node_ids = []

    for coordinate in coordinates:
        longitude, latitude = coordinate
        node_id = get_node_id(latitude, longitude)
        
        if not node_id:
            cur.execute("INSERT INTO nodes (latitude, longitude) VALUES (%s, %s) RETURNING id", (latitude, longitude))
            node_id = cur.fetchone()[0]
        
        if (route_id, node_id) not in node_ids:
            cur.execute("INSERT INTO route_way (route_id, node_id, sequence) VALUES (%s, %s, %s)", (route_id, node_id, sequence))
            sequence += 1
            node_ids.append((route_id, node_id))

    # Function to check if an edge exists and return its weight and usage_count
    def get_edge_info(node_id_start, node_id_end):
        cur.execute("SELECT weight, usage_count FROM edges WHERE node_id_start = %s AND node_id_end = %s", (node_id_start, node_id_end))
        result = cur.fetchone()
        return result if result else None

    # Function to update edge weight and usage_count
    def update_edge(node_id_start, node_id_end, weight_adjustment):
        cur.execute("UPDATE edges SET weight = weight + %s, usage_count = usage_count + 1 WHERE node_id_start = %s AND node_id_end = %s", 
                    (weight_adjustment, node_id_start, node_id_end))

    # Insert edges into the database
    for i in range(len(node_ids) - 1):
        start_node_id = node_ids[i][1]
        end_node_id = node_ids[i + 1][1]
        weight_adjustment = rating_weight_adjustment[rating]
        
        edge_info = get_edge_info(start_node_id, end_node_id)
        
        if edge_info is None:
            cur.execute("INSERT INTO edges (node_id_start, node_id_end, weight, usage_count) VALUES (%s, %s, %s, %s)", 
                        (start_node_id, end_node_id, weight_adjustment, 1))
        else:
            update_edge(start_node_id, end_node_id, weight_adjustment)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

# Call random Entries

In [24]:
num_entries = 200

for i in range(num_entries):
    # Generieren der zufälligen Start- und Endkoordinaten
    start_lat, start_lon = generate_random_coordinates()
    end_lat, end_lon = generate_random_coordinates()
    
    # Generieren des zufälligen Ratings
    rating = generate_random_rating()

    # Routeninformationen von OpenRouteService abrufen
    response = client.directions(coordinates=[[start_lon, start_lat], [end_lon, end_lat]], profile='driving-car', format='geojson')
    data = response['features'][0]['geometry']['coordinates']

    insert_route(data, start_lat, start_lon, end_lat, end_lon, rating)

    print(data)

    time.sleep(1) #Use sleep timer, because otherwise the API will block you after aroung 50 requests

[[13.419465, 52.490005], [13.419681, 52.489937], [13.420512, 52.489614], [13.420731, 52.48953], [13.421096, 52.489389], [13.421397, 52.489273], [13.421448, 52.489253], [13.421744, 52.489139], [13.421786, 52.489123], [13.422033, 52.489026], [13.422101, 52.488999], [13.423189, 52.488571], [13.423303, 52.488527], [13.423482, 52.488458], [13.423562, 52.488426], [13.424414, 52.488092], [13.424535, 52.488045], [13.424658, 52.487997], [13.425151, 52.487808], [13.425419, 52.48786], [13.42532, 52.487968], [13.42523, 52.488078], [13.425071, 52.488323], [13.424876, 52.488637], [13.42479, 52.488737], [13.424606, 52.489036], [13.424373, 52.489403], [13.424151, 52.489763], [13.424048, 52.48993], [13.423947, 52.490092], [13.42386, 52.490228], [13.423837, 52.490264], [13.423707, 52.490475], [13.423659, 52.490552], [13.423512, 52.490791], [13.423325, 52.491087], [13.423196, 52.491306], [13.423116, 52.49144], [13.423077, 52.491506], [13.422718, 52.49207], [13.422663, 52.492159], [13.422152, 52.492984], 

c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit 

[[13.417029, 52.498094], [13.417034, 52.498101], [13.417093, 52.498185], [13.417121, 52.498224], [13.417116, 52.498276], [13.417377, 52.498673], [13.417377, 52.498842], [13.417681, 52.49884], [13.418009, 52.498815], [13.418205, 52.498775], [13.418381, 52.498785], [13.418579, 52.498858], [13.418611, 52.498879], [13.418665, 52.49893], [13.41871, 52.499023], [13.41868, 52.49916], [13.4186, 52.49924], [13.418485, 52.499299], [13.418344, 52.499332], [13.418189, 52.499335], [13.41809, 52.499537], [13.418113, 52.499648], [13.418205, 52.499775], [13.418426, 52.50008], [13.418487, 52.500164], [13.418569, 52.500278], [13.418751, 52.500529], [13.418869, 52.500693], [13.418934, 52.500786], [13.419054, 52.500959], [13.419284, 52.501285], [13.419547, 52.50165], [13.419571, 52.501682], [13.419589, 52.501708], [13.419678, 52.50183], [13.419832, 52.502044], [13.419982, 52.502249], [13.420057, 52.502357], [13.420213, 52.502581], [13.420233, 52.50261], [13.420244, 52.502625], [13.42044, 52.502907], [13.4

c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


[[13.431808, 52.497428], [13.431642, 52.497493], [13.431482, 52.497556], [13.4313, 52.497627], [13.430785, 52.497828], [13.430667, 52.497872], [13.430574, 52.497964], [13.430115, 52.498135], [13.429723, 52.498273], [13.429587, 52.498318], [13.429298, 52.498415], [13.428733, 52.498604], [13.428527, 52.498673], [13.427156, 52.499127], [13.426838, 52.499222], [13.426542, 52.499269], [13.426351, 52.49926], [13.425763, 52.499236], [13.425101, 52.499221], [13.424049, 52.499208], [13.42266, 52.49919], [13.422346, 52.499188], [13.421963, 52.499185], [13.421218, 52.499178], [13.420846, 52.499175], [13.420648, 52.499173], [13.420379, 52.499171], [13.419933, 52.499167], [13.419561, 52.499162], [13.419302, 52.499159], [13.419226, 52.499157], [13.41868, 52.49916], [13.4186, 52.49924], [13.418485, 52.499299], [13.418344, 52.499332], [13.418189, 52.499335], [13.418033, 52.499305], [13.41786, 52.499207], [13.417815, 52.499151], [13.417584, 52.499045], [13.417088, 52.498973], [13.41694, 52.498954], [13

# Map Creation

In [25]:
import psycopg2
import folium
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Funktion zur Bestimmung der Farbe basierend auf dem Gewicht
def get_color(weight, vmin, vmax):
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    cmap = cm.RdYlGn  # Direkter Zugriff auf die Colormap
    return mcolors.to_hex(cmap(norm(weight)))

# Verbindung zur PostgreSQL-Datenbank
conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
cur = conn.cursor()

# Abfrage aller Edges mit deren Start- und Endkoordinaten sowie Gewichten
cur.execute("""
    SELECT ns.latitude, ns.longitude, ne.latitude, ne.longitude, e.weight
    FROM edges e
    JOIN nodes ns ON e.node_id_start = ns.id
    JOIN nodes ne ON e.node_id_end = ne.id
""")
edges = cur.fetchall()

# Bestimmen des minimalen und maximalen Gewichts
weights = [edge[4] for edge in edges]
vmin = min(weights)
vmax = max(weights)

# Karte initialisieren
map_center = [52.5, 13.4]  # Beispielzentrum (Berlin)
mymap = folium.Map(location=map_center, zoom_start=12)

# Edges zur Karte hinzufügen
for edge in edges:
    start_lat, start_lon, end_lat, end_lon, weight = edge
    color = get_color(weight, vmin, vmax)
    folium.PolyLine(
        locations=[(start_lat, start_lon), (end_lat, end_lon)],
        color=color,
        weight=5,
        opacity=0.8
    ).add_to(mymap)

# HTML-Datei speichern
mymap.save('route_map.html')

# Verbindung schließen
cur.close()
conn.close()
